In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Установка зависимостей
!pip install -q segmentation-models-pytorch
!pip install -q albumentations
!pip install -q pytorch-lightning

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


In [ ]:
# Импорты
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import transforms as T
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
from glob import glob
from tqdm import tqdm

In [ ]:
# Конфигурация
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 4
NUM_CLASSES = 19  # для Cityscapes (18 + 1 кастомный)
LR = 1e-3
EPOCHS = 25

In [ ]:
# Пути к данным
TRAIN_IMG_DIR = "/content/drive/MyDrive/data/train/images"
TRAIN_MASK_DIR = "/content/drive/MyDrive/data/train/masks"
VAL_IMG_DIR = "/content/drive/MyDrive/data/val/images"
VAL_MASK_DIR = "/content/drive/MyDrive/data/val/masks"

In [ ]:
# Аугментации
train_transform = A.Compose([
    A.Resize(256, 512),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(256, 512),
    ToTensorV2(),
])

# Класс датасета
class SegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, mask_dir, transform):
        self.image_paths = sorted(glob(os.path.join(image_dir, '*')))
        self.mask_paths = sorted(glob(os.path.join(mask_dir, '*')))
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        mask = np.array(Image.open(self.mask_paths[idx]))

        augmented = self.transform(image=image, mask=mask)
        image = augmented['image']
        mask = augmented['mask']

        return image, mask.long()

# Загрузка данных
train_dataset = SegmentationDataset(TRAIN_IMG_DIR, TRAIN_MASK_DIR, train_transform)
val_dataset = SegmentationDataset(VAL_IMG_DIR, VAL_MASK_DIR, val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Определение модели ESNet (через Smp для упрощения, как ENet или аналог)
model = smp.ESPNetV2(classes=NUM_CLASSES, activation=None).to(DEVICE)

# Оптимизатор, лосс, метрики
loss_fn = smp.losses.DiceLoss(mode='multiclass')
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# Функция обучения
def train_one_epoch(loader, model, optimizer, loss_fn):
    model.train()
    epoch_loss = 0
    for images, masks in tqdm(loader):
        images, masks = images.to(DEVICE), masks.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(loader)

# Функция валидации
def evaluate(loader, model, loss_fn):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, masks in loader:
            images, masks = images.to(DEVICE), masks.to(DEVICE)
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            val_loss += loss.item()
    return val_loss / len(loader)

# Обучение
for epoch in range(EPOCHS):
    train_loss = train_one_epoch(train_loader, model, optimizer, loss_fn)
    val_loss = evaluate(val_loader, model, loss_fn)
    scheduler.step()
    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Сохранение модели
torch.save(model.state_dict(), "esnet_segmentation.pth")


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
# Пример кода дообучения ESNet
from model import ESNet  # Ваша реализация или сторонняя
from dataset import CustomCityscapes  # с кастомным классом "courier"
import torch, torchvision
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

# Датасеты
train_set = CustomCityscapes(split='train')
val_set = CustomCityscapes(split='val')
train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
val_loader = DataLoader(val_set, batch_size=4)

# Модель
model = ESNet(num_classes=20)  # 19 классов + курьер
model = model.cuda()

# Обучение
optimizer = AdamW(model.parameters(), lr=1e-4)
criterion = CrossEntropyLoss(weight=torch.tensor([...]).cuda())  # class weights
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

for epoch in range(50):
    model.train()
    for imgs, masks in train_loader:
        imgs, masks = imgs.cuda(), masks.cuda()
        preds = model(imgs)
        loss = criterion(preds, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    scheduler.step()
    print(f"Epoch {epoch} complete.")


Custom TB Handler failed, unregistering
